In [ ]:
import json
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
with open('enbn.json', 'r') as file:
    dataenbn = json.load(file)

with open('bnen.json', 'r') as file:
    databnen = json.load(file)

with open('enhi.json', 'r') as file:
    dataenhi = json.load(file)

with open('hien.json', 'r') as file:
    datahien = json.load(file)

In [ ]:
!pip install datasets

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq

In [ ]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
source_lang = "en"
target_lang = "bn"
prefix = "translate English to Bengali: "

inputs = [prefix + example[source_lang] for example in dataenbn]
targets = [example[target_lang] for example in dataenbn]
model_inputs_en1 = tokenizer(inputs, max_length=128, truncation=True)

with tokenizer.as_target_tokenizer():
  labels_bn = tokenizer(targets, max_length=128, truncation=True)

In [ ]:
source_lang = "en"
target_lang = "hi"
prefix = "translate English to Hindi: "

inputs = [prefix + example[source_lang] for example in dataenhi]
targets = [example[target_lang] for example in dataenhi]
model_inputs_en2 = tokenizer(inputs, max_length=128, truncation=True)

with tokenizer.as_target_tokenizer():
  labels_hi = tokenizer(targets, max_length=128, truncation=True)

In [ ]:
source_lang = "hi"
target_lang = "en"
prefix = "translate Hindi to English: "

inputs = [prefix + example[source_lang] for example in datahien]
targets = [example[target_lang] for example in datahien]

with tokenizer.as_target_tokenizer():
  model_inputs_hi = tokenizer(inputs, max_length=128, truncation=True)

labels_en1 = tokenizer(targets, max_length=128, truncation=True)

In [ ]:
source_lang = "bn"
target_lang = "en"
prefix = "translate Bengali to English: "

inputs = [prefix + example[source_lang] for example in databnen]
targets = [example[target_lang] for example in databnen]

with tokenizer.as_target_tokenizer():
  model_inputs_bn = tokenizer(inputs, max_length=128, truncation=True)

labels_en2 = tokenizer(targets, max_length=128, truncation=True)

In [ ]:
# Concatenate model inputs
model_inputs = {
    "input_ids": model_inputs_en1["input_ids"] + model_inputs_en2["input_ids"] + model_inputs_hi["input_ids"] + model_inputs_bn["input_ids"],
    "attention_mask": model_inputs_en1["attention_mask"] + model_inputs_en2["attention_mask"] + model_inputs_hi["attention_mask"] + model_inputs_bn["attention_mask"]
}

# Concatenate labels
labels = labels_en1["input_ids"] + labels_hi["input_ids"] + labels_bn["input_ids"] + labels_en2["input_ids"]

# Add labels to model_inputs
model_inputs["labels"] = labels

In [ ]:
from datasets import Dataset

# Create a dictionary containing your tokenized inputs and labels
data = {
    "input_ids": model_inputs["input_ids"],
    "attention_mask": model_inputs["attention_mask"],
    "labels": model_inputs["labels"]
}

# Convert the dictionary to a dataset object
dataset = Dataset.from_dict(data)

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
print(dataset)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
!pip show transformers
!pip show accelerate

In [ ]:
!pip uninstall transformers accelerate
!pip install transformers[torch]

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define and initialize your T5 model and tokenizer
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define your training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    #save_steps=1000,
    num_train_epochs=1,
    fp16=False,
)

# Instantiate the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],  # Use the created dataset
    eval_dataset=dataset["test"],   # You can use the same dataset for evaluation as well
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start the training
trainer.train()

# Save the final model checkpoint
trainer.save_model("./final_model")


In [ ]:
with open('hibn.json', 'r') as file:
    datahibn = json.load(file)

In [ ]:
source_lang = "hi"
target_lang = "bn"
prefix = "translate Hindi to Bengali: "

inputs = [prefix + example[source_lang] for example in datahibn]
targets = [example[target_lang] for example in datahibn]

with tokenizer.as_target_tokenizer():
  model_inputs_hi = tokenizer(inputs, max_length=128, truncation=True)
with tokenizer.as_target_tokenizer():
  labels_bn = tokenizer(targets, max_length=128, truncation=True)


In [ ]:
model_inputs_hi = {
    "input_ids": model_inputs_hi["input_ids"],
    "attention_mask": model_inputs_hi["attention_mask"]
}

In [ ]:
inputs_hi = Dataset.from_dict(model_inputs_hi)

In [ ]:
print(inputs_hi['input_ids'][0])
print(inputs_hi['input_ids'][1])
print(inputs_hi['input_ids'][2])
print(inputs_hi['input_ids'][3])

In [ ]:
fine_tuned_model = MT5ForConditionalGeneration.from_pretrained("./final_model")

In [ ]:
import torch

In [ ]:
# Tokenize the preprocessed Hindi sentences
hindi_inputs = inputs_hi['input_ids']
# Determine the maximum sequence length
max_length = max(len(seq) for seq in hindi_inputs)


# Pad sequences to the maximum length
padded_inputs = [seq + [tokenizer.pad_token_id] * (max_length - len(seq)) for seq in hindi_inputs]

# Convert to tensor
hindi_inputs_tensor = torch.tensor(padded_inputs)
hindi_inputs_tensor = torch.tensor(hindi_inputs_tensor)

# Perform inference
with torch.no_grad():
    outputs = fine_tuned_model.generate(input_ids=hindi_inputs_tensor,
                             max_length=128,  # Adjust max_length as needed
                             num_beams=4,     # Adjust num_beams as needed
                             early_stopping=True,
                             num_return_sequences=1)

# Decode the model outputs to obtain translated Bengali sentences
with tokenizer.as_target_tokenizer():
  translated_bengali_sentences = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
print(translated_bengali_sentences)

In [ ]:
print(outputs)

In [ ]:
translated_bn_json = json.dumps(translated_bengali_sentences, ensure_ascii = False)

In [ ]:
# Define the path to save the JSON file
output_json_path = 'translated_bn_json.json'

# Save JSON data to a file
with open(output_json_path, 'w', encoding='utf8') as json_file:
    json_file.write(translated_bn_json)

print("Conversion completed. JSON file saved as:", output_json_path)

In [ ]:
references = [sample['bn'] for sample in datahibn]

In [ ]:
print(references)

In [ ]:
# Calculate corpus BLEU score
from nltk.translate.bleu_score import corpus_bleu
bleu_score = corpus_bleu(references, translated_bengali_sentences)
print("BLEU Score:", bleu_score)

In [ ]:
from datasets import load_metric

bleu = load_metric("bleu")

sum_bleu = 0
for sent in range(0, len(translated_bengali_sentences)):
  pred = [[i for i in translated_bengali_sentences[sent]]]
  ref = [[[i for i in references[sent]]]]
  score = bleu.compute(predictions = pred, references = ref)
  sum_bleu = sum_bleu + score['bleu']
  print(score)

print(sum_bleu)
print(sum_bleu/100)